In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

In [ ]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_dimp"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

In [ ]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

In [ ]:
# ============================================================================
# CONFIGURAÇÃO INICIAL - PROJETO DIMP
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    concat as spark_concat,
    lit as spark_lit,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    countDistinct as spark_countDistinct
)
from pyspark.sql.types import DoubleType, IntegerType

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# ✅ CORREÇÃO: Não usar abs() que conflita com PySpark
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}' if abs(x) > 0.01 else f'{x:.6f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Acesso ao Spark
spark = session.sparkSession

print("=" * 80)
print("🔍 SISTEMA DE ANÁLISE DIMP - Meios de Pagamento por CPF/CNPJ")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

In [ ]:
# ============================================================================
# PANORAMA GERAL DO SISTEMA DIMP
# ============================================================================

print("\n" + "=" * 80)
print("📊 VERIFICAÇÃO DE TABELAS E ESTATÍSTICAS GERAIS")
print("=" * 80)

# Lista de tabelas do projeto
tabelas_dimp = [
    'teste.dimp_cnpj_base',
    'teste.dimp_socios',
    'teste.dimp_pagamentos_cnpj',
    'teste.dimp_pagamentos_cpf',
    'teste.dimp_comparacao_cnpj_cpf',
    'teste.dimp_score_final',
    'teste.dimp_operacoes_suspeitas',
    'teste.dimp_socios_multiplas_empresas'
]

print("\n📋 Verificando existência e tamanho das tabelas:\n")
for tabela in tabelas_dimp:
    try:
        count = spark.sql(f"SELECT COUNT(*) as cnt FROM {tabela}").collect()[0]['cnt']
        print(f"✅ {tabela:50s} → {count:>12,} registros")
    except Exception as e:
        print(f"❌ {tabela:50s} → NÃO ENCONTRADA")

print("\n" + "=" * 80)
print("📈 ESTATÍSTICAS GERAIS DO SISTEMA")
print("=" * 80)

# Criar view temporária com estatísticas gerais
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_panorama_geral AS
SELECT 
    COUNT(DISTINCT cnpj) AS total_empresas,
    COUNT(DISTINCT CASE WHEN classificacao_risco IN ('ALTO', 'MÉDIO-ALTO') THEN cnpj END) AS empresas_suspeitas,
    CAST(COALESCE(SUM(total_geral), 0) AS DOUBLE) AS volume_total,
    CAST(COALESCE(SUM(total_recebido_cpf), 0) AS DOUBLE) AS volume_cpf,
    CAST(COALESCE(SUM(total_recebido_cnpj), 0) AS DOUBLE) AS volume_cnpj,
    CAST(COALESCE(AVG(perc_recebido_cpf), 0) AS DOUBLE) AS media_perc_cpf,
    CAST(COALESCE(AVG(score_risco_final), 0) AS DOUBLE) AS media_score,
    COUNT(DISTINCT CASE WHEN perc_recebido_cpf >= 80 THEN cnpj END) AS empresas_80pct_cpf,
    COUNT(DISTINCT CASE WHEN perc_recebido_cpf >= 50 THEN cnpj END) AS empresas_50pct_cpf
FROM teste.dimp_score_final
""")

# Verificar tamanho
total_panorama = spark.sql("SELECT COUNT(*) as cnt FROM vw_panorama_geral").collect()[0]['cnt']
print(f"\n📊 Total de registros no panorama: {total_panorama}")

# Converter para pandas (é pequeno, só 1 linha)
df_panorama = spark.sql("SELECT * FROM vw_panorama_geral").toPandas()

if len(df_panorama) > 0:
    p = df_panorama.iloc[0]
    
    print("\n🎯 MÉTRICAS PRINCIPAIS:")
    print(f"  • Total de Empresas Analisadas: {int(p['total_empresas']):,}")
    print(f"  • Empresas Suspeitas (Alto/Médio-Alto): {int(p['empresas_suspeitas']):,}")
    print(f"  • Volume Total Movimentado: R$ {p['volume_total']:,.2f}")
    print(f"  • Volume em CPF (Sócios): R$ {p['volume_cpf']:,.2f}")
    print(f"  • Volume em CNPJ (Empresa): R$ {p['volume_cnpj']:,.2f}")
    
    print(f"\n📊 INDICADORES DE RISCO:")
    print(f"  • % Médio Recebido em CPF: {p['media_perc_cpf']:.2f}%")
    print(f"  • Score Médio de Risco: {p['media_score']:.2f}")
    print(f"  • Empresas com ≥80% CPF: {int(p['empresas_80pct_cpf']):,}")
    print(f"  • Empresas com ≥50% CPF: {int(p['empresas_50pct_cpf']):,}")
    
    # Calcular % de risco
    perc_suspeitas = (int(p['empresas_suspeitas']) / int(p['total_empresas'])) * 100 if int(p['total_empresas']) > 0 else 0
    perc_cpf = (p['volume_cpf'] / p['volume_total']) * 100 if p['volume_total'] > 0 else 0
    
    print(f"\n⚠️  ALERTAS:")
    print(f"  • {perc_suspeitas:.1f}% das empresas são suspeitas")
    print(f"  • {perc_cpf:.1f}% do volume total passa por CPF de sócios")
else:
    print("⚠️ Nenhum dado encontrado no panorama geral")

print("\n" + "=" * 80)

In [ ]:
# ============================================================================
# DISTRIBUIÇÃO DE RISCO - ANÁLISE POR CLASSIFICAÇÃO
# ============================================================================

print("\n" + "=" * 80)
print("📊 DISTRIBUIÇÃO DE EMPRESAS POR CLASSIFICAÇÃO DE RISCO")
print("=" * 80)

# Criar view de distribuição
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_distribuicao_risco AS
SELECT 
    classificacao_risco,
    COUNT(DISTINCT cnpj) AS qtd_empresas,
    CAST(COALESCE(SUM(total_geral), 0) AS DOUBLE) AS volume_total,
    CAST(COALESCE(SUM(total_recebido_cpf), 0) AS DOUBLE) AS volume_cpf,
    CAST(COALESCE(AVG(perc_recebido_cpf), 0) AS DOUBLE) AS media_perc_cpf,
    CAST(COALESCE(AVG(score_risco_final), 0) AS DOUBLE) AS media_score
FROM teste.dimp_score_final
GROUP BY classificacao_risco
ORDER BY 
    CASE classificacao_risco
        WHEN 'ALTO' THEN 1
        WHEN 'MÉDIO-ALTO' THEN 2
        WHEN 'MÉDIO' THEN 3
        ELSE 4
    END
""")

# Verificar tamanho e converter
total_dist = spark.sql("SELECT COUNT(*) as cnt FROM vw_distribuicao_risco").collect()[0]['cnt']
print(f"📊 Total de classificações: {total_dist}")

df_dist = spark.sql("SELECT * FROM vw_distribuicao_risco").toPandas()

print("\n📋 Distribuição por Classificação:\n")
for idx, row in df_dist.iterrows():
    print(f"  {row['classificacao_risco']:12s} → {int(row['qtd_empresas']):>6,} empresas | "
          f"Volume: R$ {row['volume_total']:>15,.2f} | "
          f"Score Médio: {row['media_score']:>6.2f}")

# ============================================================================
# GRÁFICOS INTERATIVOS COM PLOTLY
# ============================================================================

# 1. Pizza - Distribuição de Empresas
fig_pizza = go.Figure(data=[go.Pie(
    labels=df_dist['classificacao_risco'],
    values=df_dist['qtd_empresas'],
    hole=0.4,
    marker=dict(colors=['#d62728', '#ff7f0e', '#ffdd70', '#2ca02c']),
    textinfo='label+percent+value',
    texttemplate='<b>%{label}</b><br>%{value:,} empresas<br>%{percent:.1%}'
)])

fig_pizza.update_layout(
    title='<b>Distribuição de Empresas por Nível de Risco</b>',
    height=500,
    showlegend=True,
    font=dict(size=12)
)

fig_pizza.show()

# 2. Barras Horizontais - Volume por Classificação
fig_barras = go.Figure()

fig_barras.add_trace(go.Bar(
    y=df_dist['classificacao_risco'],
    x=df_dist['volume_cpf'] / 1e6,
    name='Volume CPF',
    orientation='h',
    marker=dict(color='#ff7f0e'),
    text=df_dist['volume_cpf'].apply(lambda x: f'R$ {x/1e6:.1f}M'),
    textposition='inside'
))

fig_barras.add_trace(go.Bar(
    y=df_dist['classificacao_risco'],
    x=df_dist['volume_total'] / 1e6,
    name='Volume Total',
    orientation='h',
    marker=dict(color='#1f77b4'),
    text=df_dist['volume_total'].apply(lambda x: f'R$ {x/1e6:.1f}M'),
    textposition='inside'
))

fig_barras.update_layout(
    title='<b>Volume Financeiro por Classificação de Risco</b>',
    xaxis_title='Volume (Milhões R$)',
    yaxis_title='Classificação',
    barmode='group',
    height=500,
    showlegend=True
)

fig_barras.show()

# 3. Gauge - Score Médio Geral
score_geral = df_dist['media_score'].mean()

fig_gauge = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=score_geral,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "<b>Score Médio de Risco Geral</b>", 'font': {'size': 24}},
    delta={'reference': 50},
    gauge={
        'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "darkblue"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 40], 'color': '#2ca02c'},
            {'range': [40, 60], 'color': '#ffdd70'},
            {'range': [60, 80], 'color': '#ff7f0e'},
            {'range': [80, 100], 'color': '#d62728'}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 80
        }
    }
))

fig_gauge.update_layout(height=400)
fig_gauge.show()

print("\n✅ Gráficos interativos gerados com sucesso!")

In [ ]:
# ============================================================================
# RANKING: TOP 50 EMPRESAS COM MAIOR RISCO
# ============================================================================

print("\n" + "=" * 80)
print("🎯 TOP 50 EMPRESAS COM MAIOR SCORE DE RISCO")
print("=" * 80)

# Criar view com top 50
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_top50_suspeitas AS
SELECT 
    cnpj,
    nm_razao_social,
    regime_tributario,
    COALESCE(municipio, 'NÃO INFORMADO') AS municipio,
    COALESCE(uf, 'N/A') AS uf,
    COALESCE(nm_cnae1, 'NÃO INFORMADO') AS nm_cnae1,
    CAST(COALESCE(total_recebido_cnpj, 0) AS DOUBLE) AS total_cnpj,
    CAST(COALESCE(total_recebido_cpf, 0) AS DOUBLE) AS total_cpf,
    CAST(COALESCE(total_geral, 0) AS DOUBLE) AS total_geral,
    CAST(COALESCE(perc_recebido_cpf, 0) AS DOUBLE) AS perc_cpf,
    qtd_socios_recebendo,
    meses_com_pagto_cpf,
    CAST(COALESCE(score_risco_final, 0) AS DOUBLE) AS score_final,
    classificacao_risco,
    CAST(COALESCE(score_proporcao, 0) AS DOUBLE) AS score_proporcao,
    CAST(COALESCE(score_volume_cpf, 0) AS DOUBLE) AS score_volume,
    CAST(COALESCE(score_qtd_socios, 0) AS DOUBLE) AS score_socios,
    CAST(COALESCE(score_desvio_regime, 0) AS DOUBLE) AS score_desvio,
    CAST(COALESCE(score_consistencia, 0) AS DOUBLE) AS score_consistencia
FROM teste.dimp_score_final
WHERE classificacao_risco IN ('ALTO', 'MÉDIO-ALTO')
ORDER BY score_risco_final DESC
LIMIT 50
""")

# Verificar e converter
total_top50 = spark.sql("SELECT COUNT(*) as cnt FROM vw_top50_suspeitas").collect()[0]['cnt']
print(f"\n📊 Total de empresas no ranking: {total_top50}")

df_top50 = spark.sql("SELECT * FROM vw_top50_suspeitas").toPandas()

print(f"\n🔝 TOP 20 EMPRESAS MAIS SUSPEITAS:\n")
for idx, row in df_top50.head(20).iterrows():
    print(f"{idx+1:2d}. CNPJ: {row['cnpj']}")
    print(f"    Razão Social: {str(row['nm_razao_social'])[:60]}")
    print(f"    Score: {row['score_final']:.2f} | Risco: {row['classificacao_risco']}")
    print(f"    % CPF: {row['perc_cpf']:.1f}% | Total: R$ {row['total_geral']:,.2f}")
    
    # ✅ CORREÇÃO: Verificar se nm_cnae1 não é None antes de fatiar
    cnae_str = str(row['nm_cnae1'])[:40] if row['nm_cnae1'] is not None else 'N/A'
    print(f"    Município: {row['municipio']} - {row['uf']} | CNAE: {cnae_str}")
    print()

# ============================================================================
# HEATMAP: Composição do Score - Top 20
# ============================================================================

# Preparar matriz de scores
df_scores = df_top50.head(20)[['cnpj', 'score_proporcao', 'score_volume', 
                                'score_socios', 'score_desvio', 'score_consistencia']].copy()

df_scores['cnpj_label'] = df_scores['cnpj'].str[-4:]  # Últimos 4 dígitos
df_scores_matrix = df_scores.set_index('cnpj_label')[['score_proporcao', 'score_volume', 
                                                        'score_socios', 'score_desvio', 'score_consistencia']]

# Renomear colunas
df_scores_matrix.columns = ['Proporção\nCPF', 'Volume\nCPF', 'Qtd\nSócios', 
                            'Desvio\nRegime', 'Consistência\nTemporal']

fig_heatmap = go.Figure(data=go.Heatmap(
    z=df_scores_matrix.values.T,
    x=df_scores_matrix.index,
    y=df_scores_matrix.columns,
    colorscale='YlOrRd',
    text=df_scores_matrix.values.T,
    texttemplate='%{text:.0f}',
    textfont={"size": 10},
    colorbar=dict(title="Score")
))

fig_heatmap.update_layout(
    title='<b>Composição do Score de Risco - Top 20 Empresas</b><br><sub>Últimos 4 dígitos do CNPJ</sub>',
    xaxis_title='CNPJ (final)',
    yaxis_title='Componente do Score',
    height=500,
    font=dict(size=11)
)

fig_heatmap.show()

# ============================================================================
# GRÁFICO: Score Final - Top 30
# ============================================================================

df_top30_chart = df_top50.head(30).sort_values('score_final', ascending=True)
df_top30_chart['cnpj_curto'] = df_top30_chart['cnpj'].str[-6:]

colors_risk = ['#8b0000' if x >= 90 else '#d62728' if x >= 80 else '#ff7f0e' 
               for x in df_top30_chart['score_final']]

fig_bar_score = go.Figure(go.Bar(
    y=df_top30_chart['cnpj_curto'],
    x=df_top30_chart['score_final'],
    orientation='h',
    marker=dict(color=colors_risk),
    text=df_top30_chart['score_final'].apply(lambda x: f'{x:.1f}'),
    textposition='outside',
    hovertemplate='<b>CNPJ (final): %{y}</b><br>Score: %{x:.2f}<br><extra></extra>'
))

fig_bar_score.update_layout(
    title='<b>Score de Risco Final - Top 30 Empresas</b>',
    xaxis_title='Score de Risco',
    yaxis_title='CNPJ (6 últimos dígitos)',
    height=800,
    showlegend=False
)

fig_bar_score.show()

print("\n✅ Análise do ranking concluída!")

In [ ]:
# ============================================================================
# ANÁLISE TEMPORAL: EVOLUÇÃO DOS PAGAMENTOS
# ============================================================================

print("\n" + "=" * 80)
print("📈 ANÁLISE TEMPORAL - EVOLUÇÃO DOS PAGAMENTOS")
print("=" * 80)

# ✅ CORREÇÃO: Criar views separadas para CNPJ e CPF
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_temporal_cnpj AS
SELECT 
    referencia,
    COUNT(DISTINCT cnpj) AS qtd_empresas,
    CAST(COALESCE(SUM(vl_total), 0) AS DOUBLE) AS volume_total
FROM teste.dimp_pagamentos_cnpj
GROUP BY referencia
""")

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_temporal_cpf AS
SELECT 
    referencia,
    COUNT(DISTINCT cnpj) AS qtd_empresas,
    CAST(COALESCE(SUM(vl_total), 0) AS DOUBLE) AS volume_total
FROM teste.dimp_pagamentos_cpf
GROUP BY referencia
""")

# Juntar as duas views
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_evolucao_temporal AS
SELECT 
    COALESCE(c.referencia, p.referencia) AS referencia,
    COALESCE(c.qtd_empresas, 0) AS empresas_cnpj,
    COALESCE(p.qtd_empresas, 0) AS empresas_cpf,
    COALESCE(c.volume_total, 0) AS vol_cnpj,
    COALESCE(p.volume_total, 0) AS vol_cpf
FROM vw_temporal_cnpj c
FULL OUTER JOIN vw_temporal_cpf p ON c.referencia = p.referencia
ORDER BY referencia
""")

# Verificar tamanho
total_temporal = spark.sql("SELECT COUNT(*) as cnt FROM vw_evolucao_temporal").collect()[0]['cnt']
print(f"📊 Total de períodos analisados: {total_temporal}")

if total_temporal > 0 and total_temporal <= 100:
    df_temporal = spark.sql("SELECT * FROM vw_evolucao_temporal").toPandas()
    
    # Converter referência para datetime
    df_temporal['data'] = pd.to_datetime(df_temporal['referencia'].astype(str), format='%Y%m')
    df_temporal['mes_ano'] = df_temporal['data'].dt.strftime('%Y-%m')
    
    print(f"\n📅 Período analisado: {df_temporal['mes_ano'].min()} até {df_temporal['mes_ano'].max()}")
    print(f"📊 Total de meses: {len(df_temporal)}")
    
    # ========================================================================
    # GRÁFICO: Evolução do Volume
    # ========================================================================
    
    fig_evolucao = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Volume Financeiro Mensal', 'Quantidade de Empresas'),
        vertical_spacing=0.12,
        row_heights=[0.6, 0.4]
    )
    
    # Volume financeiro
    fig_evolucao.add_trace(
        go.Scatter(
            x=df_temporal['mes_ano'],
            y=df_temporal['vol_cnpj'] / 1e6,
            name='Volume CNPJ',
            mode='lines+markers',
            line=dict(color='#1f77b4', width=2),
            fill='tozeroy'
        ),
        row=1, col=1
    )
    
    fig_evolucao.add_trace(
        go.Scatter(
            x=df_temporal['mes_ano'],
            y=df_temporal['vol_cpf'] / 1e6,
            name='Volume CPF',
            mode='lines+markers',
            line=dict(color='#ff7f0e', width=2),
            fill='tozeroy'
        ),
        row=1, col=1
    )
    
    # Quantidade de empresas
    fig_evolucao.add_trace(
        go.Bar(
            x=df_temporal['mes_ano'],
            y=df_temporal['empresas_cnpj'],
            name='Empresas (CNPJ)',
            marker=dict(color='#1f77b4')
        ),
        row=2, col=1
    )
    
    fig_evolucao.add_trace(
        go.Bar(
            x=df_temporal['mes_ano'],
            y=df_temporal['empresas_cpf'],
            name='Empresas (CPF)',
            marker=dict(color='#ff7f0e')
        ),
        row=2, col=1
    )
    
    fig_evolucao.update_xaxes(title_text="Mês/Ano", row=2, col=1)
    fig_evolucao.update_yaxes(title_text="Volume (Milhões R$)", row=1, col=1)
    fig_evolucao.update_yaxes(title_text="Quantidade", row=2, col=1)
    
    fig_evolucao.update_layout(
        title='<b>Evolução Temporal dos Pagamentos (2024-2025)</b>',
        height=700,
        showlegend=True,
        hovermode='x unified'
    )
    
    fig_evolucao.show()
    
    # Estatísticas
    print(f"\n📊 ESTATÍSTICAS TEMPORAIS:")
    print(f"  • Volume Médio Mensal (CNPJ): R$ {df_temporal['vol_cnpj'].mean():,.2f}")
    print(f"  • Volume Médio Mensal (CPF): R$ {df_temporal['vol_cpf'].mean():,.2f}")
    
    # Crescimento (se houver dados suficientes)
    if len(df_temporal) >= 2:
        if df_temporal['vol_cpf'].iloc[0] > 0:
            crescimento_cpf = ((df_temporal['vol_cpf'].iloc[-1] / df_temporal['vol_cpf'].iloc[0]) - 1) * 100
            print(f"  • Crescimento CPF: {crescimento_cpf:.1f}%")
        
        if df_temporal['vol_cnpj'].iloc[0] > 0:
            crescimento_cnpj = ((df_temporal['vol_cnpj'].iloc[-1] / df_temporal['vol_cnpj'].iloc[0]) - 1) * 100
            print(f"  • Crescimento CNPJ: {crescimento_cnpj:.1f}%")
    
else:
    print(f"⚠️ Dados temporais não disponíveis ou muito volumosos ({total_temporal} registros)")

print("\n✅ Análise temporal concluída!")

In [ ]:
# ============================================================================
# ANÁLISE GEOGRÁFICA: DISTRIBUIÇÃO POR UF E MUNICÍPIO
# ============================================================================

print("\n" + "=" * 80)
print("🗺️  ANÁLISE GEOGRÁFICA")
print("=" * 80)

# Criar view geográfica
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_geografica AS
SELECT 
    uf,
    municipio,
    COUNT(DISTINCT cnpj) AS qtd_empresas,
    CAST(COALESCE(SUM(total_geral), 0) AS DOUBLE) AS volume_total,
    CAST(COALESCE(SUM(total_recebido_cpf), 0) AS DOUBLE) AS volume_cpf,
    CAST(COALESCE(AVG(perc_recebido_cpf), 0) AS DOUBLE) AS media_perc_cpf,
    CAST(COALESCE(AVG(score_risco_final), 0) AS DOUBLE) AS media_score,
    COUNT(DISTINCT CASE WHEN classificacao_risco = 'ALTO' THEN cnpj END) AS empresas_alto_risco
FROM teste.dimp_score_final
WHERE uf IS NOT NULL AND uf != ''
GROUP BY uf, municipio
""")

# Verificar tamanho
total_geo = spark.sql("SELECT COUNT(*) as cnt FROM vw_geografica").collect()[0]['cnt']
print(f"📊 Total de localidades: {total_geo}")

# Agregação por UF (sempre pequena)
df_uf = (spark.sql("""
    SELECT 
        uf,
        SUM(qtd_empresas) AS total_empresas,
        SUM(volume_total) AS volume_total,
        SUM(volume_cpf) AS volume_cpf,
        AVG(media_score) AS score_medio,
        SUM(empresas_alto_risco) AS total_alto_risco
    FROM vw_geografica
    GROUP BY uf
    ORDER BY volume_total DESC
""")
.limit(27)  # Máximo de UFs no Brasil
.toPandas())

print(f"\n📍 DISTRIBUIÇÃO POR ESTADO:\n")
for idx, row in df_uf.head(10).iterrows():
    perc_risco = (row['total_alto_risco'] / row['total_empresas']) * 100 if row['total_empresas'] > 0 else 0
    print(f"  {row['uf']:2s} → {int(row['total_empresas']):>5,} empresas | "
          f"Volume: R$ {row['volume_total']:>15,.2f} | "
          f"Score: {row['score_medio']:>6.2f} | "
          f"Alto Risco: {perc_risco:>5.1f}%")

# ========================================================================
# GRÁFICO: Mapa de Calor por UF
# ========================================================================

fig_mapa_uf = go.Figure(data=go.Choropleth(
    locations=df_uf['uf'],
    z=df_uf['score_medio'],
    locationmode='USA-states',  # Funciona para siglas de estados
    colorscale='YlOrRd',
    text=df_uf['uf'],
    marker_line_color='white',
    colorbar_title="Score<br>Médio"
))

fig_mapa_uf.update_layout(
    title='<b>Score Médio de Risco por Estado</b>',
    geo=dict(
        scope='south america',
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'
    ),
    height=600
)

fig_mapa_uf.show()

# ========================================================================
# Top Municípios
# ========================================================================

if total_geo <= 1000:
    df_municipios = (spark.sql("""
        SELECT *
        FROM vw_geografica
        ORDER BY volume_total DESC
        LIMIT 20
    """).toPandas())
    
    print(f"\n🏙️  TOP 20 MUNICÍPIOS POR VOLUME:\n")
    for idx, row in df_municipios.iterrows():
        print(f"  {idx+1:2d}. {row['municipio'][:30]:30s} ({row['uf']}) → "
              f"{int(row['qtd_empresas']):>4,} empresas | "
              f"R$ {row['volume_total']:>15,.2f}")
    
    # Gráfico
    fig_municipios = go.Figure(go.Bar(
        y=df_municipios['municipio'].str[:25] + ' (' + df_municipios['uf'] + ')',
        x=df_municipios['volume_total'] / 1e6,
        orientation='h',
        marker=dict(color=df_municipios['media_score'], colorscale='YlOrRd',
                   showscale=True, colorbar=dict(title="Score")),
        text=df_municipios['volume_total'].apply(lambda x: f'R$ {x/1e6:.1f}M'),
        textposition='outside'
    ))
    
    fig_municipios.update_layout(
        title='<b>Top 20 Municípios por Volume Total</b>',
        xaxis_title='Volume (Milhões R$)',
        yaxis_title='Município',
        height=600
    )
    
    fig_municipios.show()

print("\n✅ Análise geográfica concluída!")

In [ ]:
# ============================================================================
# ANÁLISE DE REDE: SÓCIOS EM MÚLTIPLAS EMPRESAS
# ============================================================================

print("\n" + "=" * 80)
print("🕸️  ANÁLISE DE REDE - Sócios com Múltiplas Empresas")
print("=" * 80)

# Criar view de sócios
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_socios_rede AS
SELECT 
    cpf_socio,
    nome_socio,
    qtd_empresas,
    CAST(COALESCE(total_recebido, 0) AS DOUBLE) AS total_recebido,
    nivel_dispersao,
    cnpjs_relacionados
FROM teste.dimp_socios_multiplas_empresas
ORDER BY qtd_empresas DESC, total_recebido DESC
LIMIT 100
""")

# Verificar e converter
total_socios = spark.sql("SELECT COUNT(*) as cnt FROM vw_socios_rede").collect()[0]['cnt']
print(f"📊 Total de sócios em múltiplas empresas: {total_socios}")

if total_socios > 0:
    df_socios = spark.sql("SELECT * FROM vw_socios_rede").toPandas()
    
    print(f"\n👥 TOP 20 SÓCIOS COM MAIOR DISPERSÃO:\n")
    for idx, row in df_socios.head(20).iterrows():
        cpf_mask = row['cpf_socio'][:3] + '.***.***-' + row['cpf_socio'][-2:] if len(str(row['cpf_socio'])) == 11 else '***'
        print(f"{idx+1:2d}. CPF: {cpf_mask} | Nome: {str(row['nome_socio'])[:40]:40s}")
        print(f"    Empresas: {int(row['qtd_empresas']):>3} | "
              f"Volume: R$ {row['total_recebido']:>15,.2f} | "
              f"Dispersão: {row['nivel_dispersao']}")
        print()
    
    # ====================================================================
    # GRÁFICOS: Análise de Rede
    # ====================================================================
    
    # 1. Distribuição de Dispersão
    dist_dispersao = df_socios['nivel_dispersao'].value_counts().sort_index()
    
    fig_dispersao = go.Figure(data=[
        go.Bar(
            x=dist_dispersao.index,
            y=dist_dispersao.values,
            marker=dict(color=['#2ca02c', '#ffdd70', '#ff7f0e', '#d62728'][:len(dist_dispersao)]),
            text=dist_dispersao.values,
            textposition='outside'
        )
    ])
    
    fig_dispersao.update_layout(
        title='<b>Distribuição de Sócios por Nível de Dispersão</b>',
        xaxis_title='Nível de Dispersão',
        yaxis_title='Quantidade de Sócios',
        height=400
    )
    
    fig_dispersao.show()
    
    # 2. Scatter: Empresas vs Volume
    fig_scatter = go.Figure(data=go.Scatter(
        x=df_socios['qtd_empresas'],
        y=df_socios['total_recebido'] / 1e3,
        mode='markers',
        marker=dict(
            size=df_socios['qtd_empresas'] * 2,
            color=df_socios['qtd_empresas'],
            colorscale='YlOrRd',
            showscale=True,
            colorbar=dict(title="Qtd<br>Empresas")
        ),
        text=df_socios['nome_socio'].str[:30],
        hovertemplate='<b>%{text}</b><br>Empresas: %{x}<br>Volume: R$ %{y:.0f}k<extra></extra>'
    ))
    
    fig_scatter.update_layout(
        title='<b>Relação: Quantidade de Empresas vs Volume Recebido</b>',
        xaxis_title='Quantidade de Empresas',
        yaxis_title='Volume Total Recebido (Mil R$)',
        height=500
    )
    
    fig_scatter.show()
    
    # Estatísticas
    print(f"\n📊 ESTATÍSTICAS DA REDE:")
    print(f"  • Total de sócios em múltiplas empresas: {len(df_socios):,}")
    print(f"  • Média de empresas por sócio: {df_socios['qtd_empresas'].mean():.1f}")
    print(f"  • Máximo de empresas: {df_socios['qtd_empresas'].max()}")
    print(f"  • Volume total da rede: R$ {df_socios['total_recebido'].sum():,.2f}")
    
else:
    print("⚠️ Nenhum sócio encontrado em múltiplas empresas")

print("\n✅ Análise de rede concluída!")

In [ ]:
# ============================================================================
# PREPARAÇÃO DE DADOS PARA MACHINE LEARNING
# ============================================================================

print("\n" + "=" * 80)
print("🤖 PREPARAÇÃO DE FEATURES PARA MACHINE LEARNING")
print("=" * 80)

# Criar dataset completo
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_ml_dataset AS
SELECT 
    cnpj,
    -- Features numéricas
    CAST(COALESCE(total_recebido_cnpj, 0) AS DOUBLE) AS feat_total_cnpj,
    CAST(COALESCE(total_recebido_cpf, 0) AS DOUBLE) AS feat_total_cpf,
    CAST(COALESCE(perc_recebido_cpf, 0) AS DOUBLE) AS feat_perc_cpf,
    CAST(COALESCE(qtd_socios_recebendo, 0) AS DOUBLE) AS feat_qtd_socios,
    CAST(COALESCE(meses_com_pagto_cnpj, 0) AS DOUBLE) AS feat_meses_cnpj,
    CAST(COALESCE(meses_com_pagto_cpf, 0) AS DOUBLE) AS feat_meses_cpf,
    CAST(COALESCE(pix_cnpj, 0) AS DOUBLE) AS feat_pix_cnpj,
    CAST(COALESCE(pix_cpf, 0) AS DOUBLE) AS feat_pix_cpf,
    CAST(COALESCE(credito_cnpj, 0) AS DOUBLE) AS feat_credito_cnpj,
    CAST(COALESCE(credito_cpf, 0) AS DOUBLE) AS feat_credito_cpf,
    CAST(COALESCE(debito_cnpj, 0) AS DOUBLE) AS feat_debito_cnpj,
    CAST(COALESCE(debito_cpf, 0) AS DOUBLE) AS feat_debito_cpf,
    
    -- Scores
    CAST(COALESCE(score_proporcao, 0) AS DOUBLE) AS score_proporcao,
    CAST(COALESCE(score_volume_cpf, 0) AS DOUBLE) AS score_volume,
    CAST(COALESCE(score_qtd_socios, 0) AS DOUBLE) AS score_socios,
    CAST(COALESCE(score_desvio_regime, 0) AS DOUBLE) AS score_desvio,
    CAST(COALESCE(score_consistencia, 0) AS DOUBLE) AS score_consistencia,
    CAST(COALESCE(score_risco_final, 0) AS DOUBLE) AS score_final,
    
    -- Features categóricas
    CASE 
        WHEN classificacao_risco = 'ALTO' THEN 3
        WHEN classificacao_risco = 'MÉDIO-ALTO' THEN 2
        WHEN classificacao_risco = 'MÉDIO' THEN 1
        ELSE 0
    END AS target_risco_nivel,
    
    CASE WHEN classificacao_risco IN ('ALTO', 'MÉDIO-ALTO') THEN 1 ELSE 0 END AS target_suspeito,
    
    regime_tributario,
    uf,
    nm_cnae1
    
FROM teste.dimp_score_final
WHERE score_risco_final IS NOT NULL
""")

# Verificar tamanho
total_ml = spark.sql("SELECT COUNT(*) as cnt FROM vw_ml_dataset").collect()[0]['cnt']
print(f"\n📊 Total de registros para ML: {total_ml:,}")

if total_ml > 0:
    print(f"📥 Carregando TODOS os {total_ml:,} registros...")
    print("⚠️  Isso pode levar alguns minutos dependendo do volume...")
    
    # Carregar dados com Spark e cachear
    df_ml_spark = spark.sql("SELECT * FROM vw_ml_dataset")
    df_ml_spark.cache()
    
    # Converter para Pandas
    print("🔄 Convertendo para Pandas...")
    df_ml = df_ml_spark.toPandas()
    
    print(f"✅ Dataset carregado: {len(df_ml):,} registros × {len(df_ml.columns)} features")
    
    # Análise da distribuição do target
    print(f"\n🎯 DISTRIBUIÇÃO DO TARGET:")
    dist_target = df_ml['target_suspeito'].value_counts()
    total_records = len(df_ml)
    print(f"  • Não Suspeitos (0): {dist_target.get(0, 0):,} ({dist_target.get(0, 0)/total_records*100:.1f}%)")
    print(f"  • Suspeitos (1): {dist_target.get(1, 0):,} ({dist_target.get(1, 0)/total_records*100:.1f}%)")
    
    # Estatísticas descritivas (SEM usar abs() que conflita com PySpark)
    print(f"\n📊 ESTATÍSTICAS DAS FEATURES PRINCIPAIS:\n")
    features_principais = ['feat_perc_cpf', 'feat_total_cpf', 'feat_qtd_socios', 
                          'score_final', 'feat_meses_cpf']
    
    # Criar descrição manual para evitar conflito
    stats_df = df_ml[features_principais].describe()
    
    # Formatar output manualmente
    print(f"{'':20s}", end="")
    for col in features_principais:
        print(f"{col:>20s}", end="")
    print()
    
    for idx in stats_df.index:
        print(f"{idx:20s}", end="")
        for col in features_principais:
            val = stats_df.loc[idx, col]
            if np.isnan(val):
                print(f"{'NaN':>20s}", end="")
            else:
                print(f"{val:>20,.2f}", end="")
        print()
    
    # Matriz de correlação
    print(f"\n🔗 CALCULANDO MATRIZ DE CORRELAÇÃO...")
    features_numericas = [col for col in df_ml.columns if col.startswith('feat_') or col.startswith('score_')]
    
    print(f"   Calculando correlações entre {len(features_numericas)} features...")
    corr_matrix = df_ml[features_numericas].corr()
    
    print(f"✅ Matriz de correlação calculada!")
    
    # Heatmap de correlação
    print(f"📊 Gerando heatmap...")
    fig_corr = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=np.round(corr_matrix.values, 2),
        texttemplate='%{text}',
        textfont={"size": 7},
        hovertemplate='%{y} vs %{x}<br>Correlação: %{z:.3f}<extra></extra>'
    ))
    
    fig_corr.update_layout(
        title='<b>Matriz de Correlação entre Features</b>',
        height=900,
        width=1000,
        xaxis=dict(tickangle=-45, tickfont=dict(size=9)),
        yaxis=dict(tickfont=dict(size=9))
    )
    
    fig_corr.show()
    
    # Top correlações com o target
    print(f"\n🎯 TOP 10 FEATURES MAIS CORRELACIONADAS COM TARGET:")
    target_corr = df_ml[features_numericas + ['target_suspeito']].corr()['target_suspeito'].drop('target_suspeito').sort_values(ascending=False)
    
    for idx, (feat, corr_val) in enumerate(target_corr.head(10).items(), 1):
        print(f"  {idx:2d}. {feat:30s} → {corr_val:+.4f}")
    
    # Distribuição das features principais
    print(f"\n📊 Gerando distribuições das features principais...")
    
    fig_dist = make_subplots(
        rows=3, cols=2,
        subplot_titles=[f.replace('feat_', '').replace('_', ' ').title() for f in features_principais[:6]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    positions = [(1,1), (1,2), (2,1), (2,2), (3,1), (3,2)]
    colors_dist = ['#1f77b4', '#ff7f0e']
    
    for idx, feat in enumerate(features_principais[:6]):
        row, col = positions[idx]
        
        # Histograma para cada classe
        for class_val, color in zip([0, 1], colors_dist):
            data_class = df_ml[df_ml['target_suspeito'] == class_val][feat]
            
            fig_dist.add_trace(
                go.Histogram(
                    x=data_class,
                    name=f"Classe {class_val}",
                    opacity=0.6,
                    marker=dict(color=color),
                    showlegend=(idx == 0)
                ),
                row=row, col=col
            )
    
    fig_dist.update_layout(
        title='<b>Distribuição das Features por Classe</b>',
        height=800,
        barmode='overlay',
        showlegend=True
    )
    
    fig_dist.show()
    
    # Salvar informações úteis
    print(f"\n💾 DATASET PREPARADO:")
    print(f"   • Variável: df_ml")
    print(f"   • Shape: {df_ml.shape}")
    print(f"   • Features numéricas: {len(features_numericas)}")
    print(f"   • Memória utilizada: {df_ml.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Verificar valores faltantes
    missing_summary = df_ml.isnull().sum()
    if missing_summary.sum() > 0:
        print(f"\n⚠️  VALORES FALTANTES ENCONTRADOS:")
        for col, missing_count in missing_summary[missing_summary > 0].items():
            print(f"   • {col}: {missing_count:,} ({missing_count/len(df_ml)*100:.2f}%)")
    else:
        print(f"\n✅ Nenhum valor faltante encontrado!")
    
else:
    print("❌ Nenhum dado disponível para ML")
    df_ml = None

print("\n" + "=" * 80)
print("✅ PREPARAÇÃO PARA ML CONCLUÍDA!")
print("=" * 80)

In [ ]:
# ============================================================================
# MACHINE LEARNING: CLUSTERING K-MEANS
# ============================================================================

print("\n" + "=" * 80)
print("🎲 APRENDIZADO NÃO SUPERVISIONADO - K-MEANS CLUSTERING")
print("=" * 80)

if df_ml is not None and len(df_ml) > 0:
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    
    # Selecionar features para clustering
    features_cluster = ['feat_perc_cpf', 'feat_total_cpf', 'feat_qtd_socios',
                       'feat_meses_cpf', 'score_proporcao', 'score_volume',
                       'score_socios', 'score_consistencia']
    
    df_cluster = df_ml[features_cluster].copy()
    df_cluster = df_cluster.fillna(0)
    
    print(f"\n📊 Features selecionadas para clustering:")
    for feat in features_cluster:
        print(f"  • {feat}")
    
    # Normalização
    print(f"\n⚙️  Normalizando features...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_cluster)
    
    # Método do Cotovelo para determinar K
    print(f"\n📈 Calculando método do cotovelo...")
    inertias = []
    K_range = range(2, 11)
    
    for k in K_range:
        kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans_temp.fit(X_scaled)
        inertias.append(kmeans_temp.inertia_)
    
    # Gráfico do cotovelo
    fig_elbow = go.Figure(data=go.Scatter(
        x=list(K_range),
        y=inertias,
        mode='lines+markers',
        marker=dict(size=10, color='#1f77b4'),
        line=dict(width=2)
    ))
    
    fig_elbow.update_layout(
        title='<b>Método do Cotovelo - Determinação do K Ideal</b>',
        xaxis_title='Número de Clusters (K)',
        yaxis_title='Inércia (Soma dos Quadrados Intra-Cluster)',
        height=400
    )
    
    fig_elbow.show()
    
    # Aplicar K-Means com k=4
    k_optimal = 4
    print(f"\n🎯 Aplicando K-Means com K={k_optimal}...")
    
    kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
    df_ml['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Análise dos clusters
    print(f"\n📊 ANÁLISE DOS CLUSTERS:\n")
    for cluster_id in range(k_optimal):
        cluster_data = df_ml[df_ml['cluster'] == cluster_id]
        print(f"Cluster {cluster_id}:")
        print(f"  • Tamanho: {len(cluster_data):,} empresas ({len(cluster_data)/len(df_ml)*100:.1f}%)")
        print(f"  • % CPF Médio: {cluster_data['feat_perc_cpf'].mean():.1f}%")
        print(f"  • Volume CPF Médio: R$ {cluster_data['feat_total_cpf'].mean():,.2f}")
        print(f"  • Score Médio: {cluster_data['score_final'].mean():.2f}")
        print(f"  • % Suspeitos: {(cluster_data['target_suspeito'].sum()/len(cluster_data)*100):.1f}%")
        print()
    
    # PCA para visualização 2D
    print(f"🔄 Reduzindo dimensionalidade com PCA...")
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    df_ml['pca1'] = X_pca[:, 0]
    df_ml['pca2'] = X_pca[:, 1]
    
    print(f"✅ Variância explicada: {pca.explained_variance_ratio_.sum()*100:.1f}%")
    
    # Visualização dos clusters
    fig_clusters = go.Figure()
    
    colors_cluster = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for cluster_id in range(k_optimal):
        cluster_data = df_ml[df_ml['cluster'] == cluster_id]
        
        fig_clusters.add_trace(go.Scatter(
            x=cluster_data['pca1'],
            y=cluster_data['pca2'],
            mode='markers',
            name=f'Cluster {cluster_id}',
            marker=dict(
                size=6,
                color=colors_cluster[cluster_id],
                opacity=0.6
            ),
            text=cluster_data['cnpj'],
            hovertemplate='<b>Cluster %{fullData.name}</b><br>CNPJ: %{text}<br>PCA1: %{x:.2f}<br>PCA2: %{y:.2f}<extra></extra>'
        ))
    
    # Centroides
    centroids_pca = pca.transform(kmeans.cluster_centers_)
    fig_clusters.add_trace(go.Scatter(
        x=centroids_pca[:, 0],
        y=centroids_pca[:, 1],
        mode='markers',
        name='Centroides',
        marker=dict(
            size=15,
            color='black',
            symbol='x',
            line=dict(width=2, color='white')
        )
    ))
    
    fig_clusters.update_layout(
        title='<b>Visualização dos Clusters (PCA 2D)</b>',
        xaxis_title=f'Componente Principal 1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
        yaxis_title=f'Componente Principal 2 ({pca.explained_variance_ratio_[1]*100:.1f}%)',
        height=600,
        showlegend=True
    )
    
    fig_clusters.show()
    
    # Perfil dos clusters
    fig_perfil = go.Figure()
    
    features_perfil = ['feat_perc_cpf', 'feat_total_cpf', 'feat_qtd_socios', 'score_final']
    df_perfil = df_ml.groupby('cluster')[features_perfil].mean()
    
    # Normalizar para visualização
    df_perfil_norm = (df_perfil - df_perfil.min()) / (df_perfil.max() - df_perfil.min())
    
    for cluster_id in range(k_optimal):
        fig_perfil.add_trace(go.Scatterpolar(
            r=df_perfil_norm.loc[cluster_id].values,
            theta=features_perfil,
            fill='toself',
            name=f'Cluster {cluster_id}',
            line=dict(color=colors_cluster[cluster_id])
        ))
    
    fig_perfil.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
        title='<b>Perfil dos Clusters (Normalizado)</b>',
        height=500,
        showlegend=True
    )
    
    fig_perfil.show()
    
else:
    print("❌ Dataset não disponível para clustering")

print("\n✅ Clustering K-Means concluído!")

In [ ]:
# ============================================================================
# MACHINE LEARNING: RANDOM FOREST CLASSIFIER
# ============================================================================

print("\n" + "=" * 80)
print("🌲 APRENDIZADO SUPERVISIONADO - RANDOM FOREST")
print("=" * 80)

if df_ml is not None and len(df_ml) > 0:
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
    
    # Preparar features e target
    features_rf = ['feat_perc_cpf', 'feat_total_cpf', 'feat_qtd_socios',
                   'feat_meses_cpf', 'feat_pix_cpf', 'feat_credito_cpf',
                   'score_proporcao', 'score_volume', 'score_socios',
                   'score_desvio', 'score_consistencia']
    
    X = df_ml[features_rf].fillna(0)
    y = df_ml['target_suspeito']
    
    print(f"\n📊 Dataset para treinamento:")
    print(f"  • Features: {len(features_rf)}")
    print(f"  • Amostras: {len(X):,}")
    print(f"  • Distribuição: {y.value_counts().to_dict()}")
    
    # Split treino/teste
    print(f"\n✂️  Dividindo em treino (70%) e teste (30%)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"  • Treino: {len(X_train):,} amostras")
    print(f"  • Teste: {len(X_test):,} amostras")
    
    # Treinar modelo
    print(f"\n🌲 Treinando Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    )
    
    rf_model.fit(X_train, y_train)
    print(f"✅ Modelo treinado!")
    
    # Predições
    y_pred = rf_model.predict(X_test)
    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]
    
    # Métricas
    print(f"\n📊 MÉTRICAS DE DESEMPENHO:\n")
    print(classification_report(y_test, y_pred, target_names=['Não Suspeito', 'Suspeito']))
    
    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f"\n🎯 AUC-ROC Score: {auc_score:.4f}")
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    
    fig_cm = go.Figure(data=go.Heatmap(
        z=cm,
        x=['Não Suspeito', 'Suspeito'],
        y=['Não Suspeito', 'Suspeito'],
        text=cm,
        texttemplate='%{text}',
        textfont={"size": 16},
        colorscale='Blues'
    ))
    
    fig_cm.update_layout(
        title='<b>Matriz de Confusão - Random Forest</b>',
        xaxis_title='Predição',
        yaxis_title='Real',
        height=400
    )
    
    fig_cm.show()
    
    # Curva ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    
    fig_roc = go.Figure()
    
    fig_roc.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'Random Forest (AUC = {auc_score:.3f})',
        line=dict(color='#1f77b4', width=2)
    ))
    
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Baseline (Random)',
        line=dict(color='red', width=1, dash='dash')
    ))
    
    fig_roc.update_layout(
        title='<b>Curva ROC - Random Forest</b>',
        xaxis_title='Taxa de Falsos Positivos',
        yaxis_title='Taxa de Verdadeiros Positivos',
        height=500,
        showlegend=True
    )
    
    fig_roc.show()
    
    # Importância das features
    feature_importance = pd.DataFrame({
        'feature': features_rf,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 IMPORTÂNCIA DAS FEATURES:\n")
    # ✅ CORREÇÃO: Imprimir manualmente para evitar conflito com pd.set_option
    for idx, row in feature_importance.iterrows():
        print(f"  {row['feature']:30s} → {row['importance']:.6f}")
    
    fig_importance = go.Figure(go.Bar(
        y=feature_importance['feature'],
        x=feature_importance['importance'],
        orientation='h',
        marker=dict(color=feature_importance['importance'], colorscale='Viridis'),
        text=feature_importance['importance'].apply(lambda x: f'{x:.4f}'),
        textposition='outside'
    ))
    
    fig_importance.update_layout(
        title='<b>Importância das Features - Random Forest</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=500
    )
    
    fig_importance.show()
    
    # Adicionar probabilidades ao dataset original
    df_ml['rf_probability'] = np.nan
    df_ml.loc[X_test.index, 'rf_probability'] = y_pred_proba
    
    print(f"\n💾 Probabilidades adicionadas ao dataset 'df_ml'")
    
else:
    print("❌ Dataset não disponível para Random Forest")

print("\n✅ Random Forest concluído!")

In [ ]:
# ============================================================================
# MACHINE LEARNING: XGBOOST CLASSIFIER
# ============================================================================

print("\n" + "=" * 80)
print("⚡ APRENDIZADO SUPERVISIONADO - XGBOOST")
print("=" * 80)

if df_ml is not None and len(df_ml) > 0:
    import xgboost as xgb
    from sklearn.metrics import classification_report, roc_auc_score, roc_curve
    
    # Usar as mesmas features do Random Forest
    X = df_ml[features_rf].fillna(0)
    y = df_ml['target_suspeito']
    
    # Usar o mesmo split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"\n⚙️  Configurando XGBoost...")
    
    # Calcular scale_pos_weight para desbalanceamento
    scale_weight = (y_train == 0).sum() / (y_train == 1).sum()
    print(f"  • Scale Pos Weight: {scale_weight:.2f}")
    
    # Treinar modelo
    print(f"\n⚡ Treinando XGBoost...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_weight,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_model.fit(X_train, y_train)
    print(f"✅ Modelo treinado!")
    
    # Predições
    y_pred_xgb = xgb_model.predict(X_test)
    y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    
    # Métricas
    print(f"\n📊 MÉTRICAS DE DESEMPENHO - XGBOOST:\n")
    print(classification_report(y_test, y_pred_xgb, target_names=['Não Suspeito', 'Suspeito']))
    
    auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
    print(f"\n🎯 AUC-ROC Score: {auc_xgb:.4f}")
    
    # Comparação RF vs XGBoost
    print(f"\n📊 COMPARAÇÃO DE MODELOS:")
    print(f"  • Random Forest AUC: {auc_score:.4f}")
    print(f"  • XGBoost AUC: {auc_xgb:.4f}")
    print(f"  • Diferença: {np.abs(auc_xgb - auc_score):.4f}")
    
    if auc_xgb > auc_score:
        print(f"  ✅ XGBoost é {((auc_xgb/auc_score - 1)*100):.2f}% melhor")
    else:
        print(f"  ✅ Random Forest é {((auc_score/auc_xgb - 1)*100):.2f}% melhor")
    
    # Curvas ROC comparadas
    fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
    
    fig_compare = go.Figure()
    
    fig_compare.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'Random Forest (AUC = {auc_score:.3f})',
        line=dict(color='#1f77b4', width=2)
    ))
    
    fig_compare.add_trace(go.Scatter(
        x=fpr_xgb, y=tpr_xgb,
        mode='lines',
        name=f'XGBoost (AUC = {auc_xgb:.3f})',
        line=dict(color='#ff7f0e', width=2)
    ))
    
    fig_compare.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Baseline (Random)',
        line=dict(color='red', width=1, dash='dash')
    ))
    
    fig_compare.update_layout(
        title='<b>Comparação: Random Forest vs XGBoost</b>',
        xaxis_title='Taxa de Falsos Positivos',
        yaxis_title='Taxa de Verdadeiros Positivos',
        height=500,
        showlegend=True
    )
    
    fig_compare.show()
    
    # Matriz de confusão XGBoost
    cm_xgb = confusion_matrix(y_test, y_pred_xgb)
    
    fig_cm_xgb = go.Figure(data=go.Heatmap(
        z=cm_xgb,
        x=['Não Suspeito', 'Suspeito'],
        y=['Não Suspeito', 'Suspeito'],
        text=cm_xgb,
        texttemplate='%{text}',
        textfont={"size": 16},
        colorscale='Oranges'
    ))
    
    fig_cm_xgb.update_layout(
        title='<b>Matriz de Confusão - XGBoost</b>',
        xaxis_title='Predição',
        yaxis_title='Real',
        height=400
    )
    
    fig_cm_xgb.show()
    
    # Importância das features - XGBoost
    feature_importance_xgb = pd.DataFrame({
        'feature': features_rf,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 IMPORTÂNCIA DAS FEATURES - XGBOOST:\n")
    for idx, row in feature_importance_xgb.iterrows():
        print(f"  {row['feature']:30s} → {row['importance']:.6f}")
    
    fig_importance_xgb = go.Figure(go.Bar(
        y=feature_importance_xgb['feature'],
        x=feature_importance_xgb['importance'],
        orientation='h',
        marker=dict(color=feature_importance_xgb['importance'], colorscale='Plasma'),
        text=feature_importance_xgb['importance'].apply(lambda x: f'{x:.4f}'),
        textposition='outside'
    ))
    
    fig_importance_xgb.update_layout(
        title='<b>Importância das Features - XGBoost</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=500
    )
    
    fig_importance_xgb.show()
    
    # Comparação de importância
    fig_comp_importance = go.Figure()
    
    fig_comp_importance.add_trace(go.Bar(
        name='Random Forest',
        y=feature_importance['feature'],
        x=feature_importance['importance'],
        orientation='h',
        marker=dict(color='#1f77b4')
    ))
    
    fig_comp_importance.add_trace(go.Bar(
        name='XGBoost',
        y=feature_importance_xgb['feature'],
        x=feature_importance_xgb['importance'],
        orientation='h',
        marker=dict(color='#ff7f0e')
    ))
    
    fig_comp_importance.update_layout(
        title='<b>Comparação: Importância das Features (RF vs XGBoost)</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=600,
        barmode='group'
    )
    
    fig_comp_importance.show()
    
# Adicionar probabilidades XGBoost ao dataset
    df_ml['xgb_probability'] = np.nan
    df_ml.loc[X_test.index, 'xgb_probability'] = y_pred_proba_xgb
    
    # Ensemble (média das probabilidades)
    df_ml['ensemble_probability'] = (df_ml['rf_probability'] + df_ml['xgb_probability']) / 2
    
    print(f"\n💾 Probabilidades XGBoost e Ensemble adicionadas ao dataset 'df_ml'")
    
    # Análise do Ensemble
    y_pred_ensemble = (df_ml.loc[X_test.index, 'ensemble_probability'] >= 0.5).astype(int)
    auc_ensemble = roc_auc_score(y_test, df_ml.loc[X_test.index, 'ensemble_probability'])
    
    print(f"\n🎯 COMPARAÇÃO FINAL DOS MODELOS:")
    print(f"  • Random Forest AUC: {auc_score:.4f}")
    print(f"  • XGBoost AUC: {auc_xgb:.4f}")
    print(f"  • Ensemble (Média) AUC: {auc_ensemble:.4f}")
    
    # ✅ CORREÇÃO: Usar max do Python explicitamente
    import builtins
    modelos_comparacao = [
        ('Random Forest', auc_score), 
        ('XGBoost', auc_xgb), 
        ('Ensemble', auc_ensemble)
    ]
    best_model = builtins.max(modelos_comparacao, key=lambda x: x[1])
    print(f"\n🏆 MELHOR MODELO: {best_model[0]} (AUC = {best_model[1]:.4f})")
    
    # ⚠️ ALERTA: AUC = 1.0 indica possível overfitting!
    if auc_score >= 0.99 or auc_xgb >= 0.99:
        print(f"\n⚠️  ATENÇÃO: AUC perfeito ou próximo de 1.0 detectado!")
        print(f"    Isso pode indicar:")
        print(f"    • Overfitting nos dados de treino")
        print(f"    • Vazamento de dados (data leakage)")
        print(f"    • Features que contêm informação do target")
        print(f"\n    💡 RECOMENDAÇÕES:")
        print(f"    1. Revisar as features usadas")
        print(f"    2. Verificar se score_final ou componentes derivam do target")
        print(f"    3. Usar apenas features independentes (feat_*)")
        print(f"    4. Aplicar validação cruzada")
    
else:
    print("❌ Dataset não disponível para XGBoost")

print("\n" + "=" * 80)
print("✅ XGBoost concluído!")
print("=" * 80)

In [ ]:
# ============================================================================
# RETREINAMENTO - CORRIGINDO DATA LEAKAGE
# ============================================================================

print("\n" + "=" * 80)
print("🔧 RETREINAMENTO DOS MODELOS - SEM DATA LEAKAGE")
print("=" * 80)

if df_ml is not None and len(df_ml) > 0:
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.ensemble import RandomForestClassifier
    import xgboost as xgb
    from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
    
    print("\n⚠️  PROBLEMA IDENTIFICADO:")
    print("   Os scores (score_proporcao, score_volume, etc.) foram calculados")
    print("   a partir do target (classificacao_risco), causando data leakage!")
    print("\n✅ SOLUÇÃO:")
    print("   Usar apenas features independentes (feat_*)")
    
    # ✅ Features INDEPENDENTES (sem data leakage)
    features_clean = [
        'feat_perc_cpf',
        'feat_total_cpf',
        'feat_total_cnpj',
        'feat_qtd_socios',
        'feat_meses_cpf',
        'feat_meses_cnpj',
        'feat_pix_cpf',
        'feat_pix_cnpj',
        'feat_credito_cpf',
        'feat_credito_cnpj',
        'feat_debito_cpf',
        'feat_debito_cnpj'
    ]
    
    X_clean = df_ml[features_clean].fillna(0)
    y_clean = df_ml['target_suspeito']
    
    print(f"\n📊 Dataset limpo:")
    print(f"  • Features independentes: {len(features_clean)}")
    print(f"  • Amostras: {len(X_clean):,}")
    
    # Split
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42, stratify=y_clean
    )
    
    # ========================================================================
    # RANDOM FOREST - Retreinamento
    # ========================================================================
    
    print(f"\n🌲 Treinando Random Forest (sem leakage)...")
    rf_clean = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    )
    
    rf_clean.fit(X_train_c, y_train_c)
    y_pred_rf_c = rf_clean.predict(X_test_c)
    y_proba_rf_c = rf_clean.predict_proba(X_test_c)[:, 1]
    auc_rf_clean = roc_auc_score(y_test_c, y_proba_rf_c)
    
    print(f"✅ Random Forest treinado! AUC = {auc_rf_clean:.4f}")
    
    # Validação cruzada
    print(f"\n🔄 Validação cruzada (5-fold)...")
    cv_scores_rf = cross_val_score(rf_clean, X_clean, y_clean, cv=5, scoring='roc_auc', n_jobs=-1)
    print(f"   AUC médio CV: {cv_scores_rf.mean():.4f} (±{cv_scores_rf.std():.4f})")
    
    # ========================================================================
    # XGBOOST - Retreinamento
    # ========================================================================
    
    print(f"\n⚡ Treinando XGBoost (sem leakage)...")
    scale_weight_c = (y_train_c == 0).sum() / (y_train_c == 1).sum()
    
    xgb_clean = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_weight_c,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_clean.fit(X_train_c, y_train_c)
    y_pred_xgb_c = xgb_clean.predict(X_test_c)
    y_proba_xgb_c = xgb_clean.predict_proba(X_test_c)[:, 1]
    auc_xgb_clean = roc_auc_score(y_test_c, y_proba_xgb_c)
    
    print(f"✅ XGBoost treinado! AUC = {auc_xgb_clean:.4f}")
    
    # Validação cruzada
    print(f"\n🔄 Validação cruzada (5-fold)...")
    cv_scores_xgb = cross_val_score(xgb_clean, X_clean, y_clean, cv=5, scoring='roc_auc', n_jobs=-1)
    print(f"   AUC médio CV: {cv_scores_xgb.mean():.4f} (±{cv_scores_xgb.std():.4f})")
    
    # ========================================================================
    # COMPARAÇÃO: COM vs SEM Data Leakage
    # ========================================================================
    
    print(f"\n" + "=" * 80)
    print("📊 COMPARAÇÃO: MODELOS COM DATA LEAKAGE vs MODELOS LIMPOS")
    print("=" * 80)
    print(f"\n{'Modelo':<25s} {'AUC (com leakage)':<20s} {'AUC (limpo)':<15s}")
    print("-" * 80)
    print(f"{'Random Forest':<25s} {auc_score:>15.4f}     {auc_rf_clean:>15.4f}")
    print(f"{'XGBoost':<25s} {auc_xgb:>15.4f}     {auc_xgb_clean:>15.4f}")
    print("-" * 80)
    
    # ========================================================================
    # MÉTRICAS DETALHADAS
    # ========================================================================
    
    print(f"\n📊 MÉTRICAS DETALHADAS - RANDOM FOREST (LIMPO):\n")
    print(classification_report(y_test_c, y_pred_rf_c, target_names=['Não Suspeito', 'Suspeito']))
    
    print(f"\n📊 MÉTRICAS DETALHADAS - XGBOOST (LIMPO):\n")
    print(classification_report(y_test_c, y_pred_xgb_c, target_names=['Não Suspeito', 'Suspeito']))
    
    # ========================================================================
    # GRÁFICOS COMPARATIVOS
    # ========================================================================
    
    # Curvas ROC
    fpr_rf_c, tpr_rf_c, _ = roc_curve(y_test_c, y_proba_rf_c)
    fpr_xgb_c, tpr_xgb_c, _ = roc_curve(y_test_c, y_proba_xgb_c)
    
    fig_roc_clean = go.Figure()
    
    fig_roc_clean.add_trace(go.Scatter(
        x=fpr_rf_c, y=tpr_rf_c,
        mode='lines',
        name=f'Random Forest (AUC={auc_rf_clean:.3f})',
        line=dict(color='#1f77b4', width=2)
    ))
    
    fig_roc_clean.add_trace(go.Scatter(
        x=fpr_xgb_c, y=tpr_xgb_c,
        mode='lines',
        name=f'XGBoost (AUC={auc_xgb_clean:.3f})',
        line=dict(color='#ff7f0e', width=2)
    ))
    
    fig_roc_clean.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Baseline',
        line=dict(color='red', width=1, dash='dash')
    ))
    
    fig_roc_clean.update_layout(
        title='<b>Curvas ROC - Modelos Limpos (Sem Data Leakage)</b>',
        xaxis_title='Taxa de Falsos Positivos',
        yaxis_title='Taxa de Verdadeiros Positivos',
        height=500
    )
    
    fig_roc_clean.show()
    
    # Importância das features
    importance_rf_c = pd.DataFrame({
        'feature': features_clean,
        'importance': rf_clean.feature_importances_
    }).sort_values('importance', ascending=False)
    
    importance_xgb_c = pd.DataFrame({
        'feature': features_clean,
        'importance': xgb_clean.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 TOP 10 FEATURES MAIS IMPORTANTES:\n")
    print("Random Forest:")
    for idx, row in importance_rf_c.head(10).iterrows():
        print(f"  {row['feature']:30s} → {row['importance']:.6f}")
    
    print("\nXGBoost:")
    for idx, row in importance_xgb_c.head(10).iterrows():
        print(f"  {row['feature']:30s} → {row['importance']:.6f}")
    
    # Gráfico comparativo de importância
    fig_importance_comp = go.Figure()
    
    fig_importance_comp.add_trace(go.Bar(
        name='Random Forest',
        y=importance_rf_c['feature'],
        x=importance_rf_c['importance'],
        orientation='h',
        marker=dict(color='#1f77b4')
    ))
    
    fig_importance_comp.add_trace(go.Bar(
        name='XGBoost',
        y=importance_xgb_c['feature'],
        x=importance_xgb_c['importance'],
        orientation='h',
        marker=dict(color='#ff7f0e')
    ))
    
    fig_importance_comp.update_layout(
        title='<b>Importância das Features - Modelos Limpos</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=600,
        barmode='group'
    )
    
    fig_importance_comp.show()
    
    # Salvar modelos limpos
    df_ml['rf_clean_probability'] = np.nan
    df_ml.loc[X_test_c.index, 'rf_clean_probability'] = y_proba_rf_c
    
    df_ml['xgb_clean_probability'] = np.nan
    df_ml.loc[X_test_c.index, 'xgb_clean_probability'] = y_proba_xgb_c
    
    print(f"\n💾 Probabilidades dos modelos limpos adicionadas ao dataset!")
    
    # Melhor modelo limpo
    import builtins
    modelos_limpos = [
        ('Random Forest', auc_rf_clean, cv_scores_rf.mean()),
        ('XGBoost', auc_xgb_clean, cv_scores_xgb.mean())
    ]
    best_clean = builtins.max(modelos_limpos, key=lambda x: x[2])  # Por CV score
    
    print(f"\n🏆 MELHOR MODELO (baseado em CV): {best_clean[0]}")
    print(f"   AUC Test: {best_clean[1]:.4f}")
    print(f"   AUC CV: {best_clean[2]:.4f}")

else:
    print("❌ Dataset não disponível")

print("\n" + "=" * 80)
print("✅ RETREINAMENTO CONCLUÍDO!")
print("=" * 80)